# Canonical statistical model for light nuclei at the LHC

This notebook implements the canonical statistical model calculations of various light (anti-)nuclei-to-proton ratios as a function charged pion multiplicity at the LHC, as reported in the paper:

- V. Vovchenko, B. Dönigus, H. Stoecker, *Multiplicity dependence of light nuclei production at LHC energies in the canonical statistical model*, [Phys. Lett. B **785**, 171 (2018)](https://doi.org/10.1016/j.physletb.2018.08.041), [arXiv:1808.05245 [hep-ph]](https://arxiv.org/abs/1808.05245)

In [1]:
#include "inc/loadFISTlibs.h"

###############################################################################
#                                                                             #
# This is Thermal-FIST version 1.2.2                                          #
#                                                                             #
# Copyright (c) 2019 Volodymyr Vovchenko <vovchenko@fias.uni-frankfurt.de>    #
#                                                                             #
# Distributed under the GNU General Public License 3.0 (GPLv3 or later)       #
#                                                                             #
# Please cite when using this code:                                           #
# V. Vovchenko, H. Stoecker, arXiv:1901.05249 [nucl-th]                       #
#                                                                             #
# The latest version is available at https://github.com/vlvovch/Thermal-FIST  #
#                                       

In [2]:
using namespace std;

In [3]:
using namespace thermalfist;

In [4]:
bool useWidth  = true;
bool useQStats = false; // Maxwel-Boltzmann statistics used in 1808.05245

In [5]:
// Particle list
ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list-withnuclei.dat");
//ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list.dat");

In [6]:
// Fill the list of the considered yield ratios

vector<int> pdgs1, pdgs2;
vector<string> names1, names2;

names1.push_back("d");
names2.push_back("p");
pdgs1.push_back(1000010020);
pdgs2.push_back(2212);

names1.push_back("He3");
names2.push_back("p");
pdgs1.push_back(1000020030);
pdgs2.push_back(2212);

names1.push_back("H3La");
names2.push_back("p");
pdgs1.push_back(1010010030);
pdgs2.push_back(2212);

names1.push_back("He4");
names2.push_back("p");
pdgs1.push_back(1000020040);
pdgs2.push_back(2212);

In [7]:
// Computes limiting GCE values
void ComputeGCERatios(vector<double>& ratiosGCE, double Tch = 0.155) 
{
    ratiosGCE.resize(0);
    
    ThermalModelIdeal modelgce(&parts);

    if (useWidth)
        modelgce.SetUseWidth(ThermalParticle::eBW);
    else
        modelgce.SetUseWidth(ThermalParticle::ZeroWidth);
    modelgce.SetStatistics(useQStats);
    // Include quantum statistics for mesons only
    if (useQStats){
        for (int i = 0; i < modelgce.TPS()->Particles().size(); ++i) {
            ThermalParticle &part = modelgce.TPS()->Particle(i);
            if (part.BaryonCharge() != 0)
                part.UseStatistics(false);
        }
    }

    modelgce.SetTemperature(Tch);
    modelgce.SetBaryonChemicalPotential(0.);
    modelgce.SetElectricChemicalPotential(0.);
    modelgce.SetStrangenessChemicalPotential(0.);

    modelgce.FillChemicalPotentials();

    modelgce.CalculateDensities();

    for (int i = 0; i < names1.size(); ++i) {
        ratiosGCE.push_back(modelgce.GetDensity(pdgs1[i], 1) / modelgce.GetDensity(pdgs2[i], 1));
    }
}

In [8]:
// Prepare the output (on the screen and to a file)

vector<std::ostream*> outs;
outs.push_back(&std::cout);

outs.push_back(NULL);

In [9]:
// Routine performing the (logarithmic scale) scan over Vc and dNpi/dy.
// Parameter k determines Vc = k dV/dy

void PerformScan(ThermalModelBase *model, 
                   double k,
                   double Tch,
                   const string& filename)
{
    ofstream fout(filename);
    outs[1] = &fout;
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "dNpipm/dy";
        *out << setw(15) << "dV/dy[fm3]";
        *out << setw(15) << "Vc[fm3]";

        for (int i = 0; i < names1.size(); ++i) {
            *out << setw(15) << (names1[i] + "/" + names2[i]);
        }

        *out << endl;
    }
    
    vector<double> ratiosGCE;
    ComputeGCERatios(ratiosGCE, Tch);
    
    model->SetTemperature(Tch);
    
    double Vmin = 1.;
    double Vmax = 15000.;
    int iters = 30;
    double logVmin = log10(Vmin), logVmax = log10(Vmax);
    double dlogV = (logVmax - logVmin) / (iters - 1);
    
    for (double logV = logVmin; logV <= logVmax + 1.e-3; logV += dlogV) {
        double V = pow(10., logV);

        model->SetVolume(V);
        model->SetCanonicalVolume(k * V);
        model->CalculateDensities();

        for(std::ostream* out : outs) {
            *out << setw(15) << 2. * model->GetDensity(211, 1) * V; // Charged pions dN/dy
            *out << setw(15) << V; // dVdy
            *out << setw(15) << k * V; // Vc

            for (int i = 0; i < names1.size(); ++i) {
                *out << setw(15) << model->GetDensity(pdgs1[i], 1) / model->GetDensity(pdgs2[i], 1);
            }

            *out << endl;
        }
    }
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "HML";
        *out << setw(15) << "HML";
        *out << setw(15) << "HML";

        for (int i = 0; i < ratiosGCE.size(); ++i) {
            *out << setw(15) << ratiosGCE[i];
        }

        *out << endl;
    }
    
    fout.close();
}

In [10]:
// Prepare the thermal model for scans
ThermalModelCanonical modelce(&parts);

if (useWidth)
    modelce.SetUseWidth(ThermalParticle::eBW);
else
    modelce.SetUseWidth(ThermalParticle::ZeroWidth);

modelce.SetStatistics(useQStats);
// Include quantum statistics for mesons only
if (useQStats){
    for (int i = 0; i < modelce.TPS()->Particles().size(); ++i) {
        ThermalParticle &part = modelce.TPS()->Particle(i);
        if (part.BaryonCharge() != 0)
            part.UseStatistics(false);
    }
}

modelce.SetBaryonChemicalPotential(0.);
modelce.SetElectricChemicalPotential(0.);
modelce.SetStrangenessChemicalPotential(0.);

modelce.SetBaryonCharge(0);
modelce.SetElectricCharge(0);
modelce.SetStrangeness(0);

modelce.FillChemicalPotentials();

In [11]:
// Tch = 155 MeV, Vc = dV/dy
PerformScan(&modelce, 1., 0.155, "out/1808.05245.Vc1.Tch155.dat");

      dNpipm/dy     dV/dy[fm3]        Vc[fm3]            d/p          He3/p         H3La/p          He4/p
      0.0766118              1              1    4.39316e-05    7.18969e-10    2.74001e-10    1.04936e-14
       0.111174        1.39317        1.39317     5.3271e-05    1.07856e-09    4.52602e-10    1.83853e-14
       0.163537        1.94091        1.94091    6.65589e-05    1.72585e-09     7.7999e-10    3.51208e-14
        0.24466        2.70402        2.70402    8.55916e-05    2.93138e-09    1.39649e-09    7.33834e-14
       0.373351        3.76715        3.76715    0.000113073    5.25294e-09    2.58693e-09    1.67317e-13
       0.581907        5.24826        5.24826    0.000153105    9.87053e-09    4.93896e-09    4.13383e-13
       0.924051        7.31171        7.31171    0.000211909    1.93339e-08    9.67768e-09    1.09583e-12
        1.48162        10.1864        10.1864    0.000298724    3.92104e-08    1.93529e-08    3.07929e-12
        2.36001        14.1914        14.1914 

In [12]:
// Tch = 155 MeV, Vc = 3dV/dy
PerformScan(&modelce, 3., 0.155, "out/1808.05245.Vc3.Tch155.dat");

      dNpipm/dy     dV/dy[fm3]        Vc[fm3]            d/p          He3/p         H3La/p          He4/p
      0.0928902              1              3     9.3132e-05    3.49993e-09    1.68812e-09    9.41377e-14
        0.14269        1.39317         4.1795     0.00012402    6.36975e-09    3.15818e-09    2.20307e-13
       0.223818        1.94091        5.82274    0.000169141    1.21337e-08    6.08185e-09     5.5709e-13
       0.356972        2.70402        8.11205    0.000235558    2.40468e-08     1.2002e-08    1.50615e-12
       0.572334        3.76715        11.3014    0.000333638    4.92144e-08    2.41157e-08    4.29739e-12
       0.906276        5.24826        15.7448    0.000477543    1.02861e-07    4.88451e-08    1.27191e-11
        1.39585        7.31171        21.9351    0.000684023     2.1588e-07    9.82739e-08    3.81465e-11
          2.086        10.1864        30.5593    0.000967632    4.44788e-07    1.92794e-07    1.12526e-10
        3.04719        14.1914        42.5742 

In [13]:
// Tch = 170 MeV, Vc = dV/dy
PerformScan(&modelce, 1., 0.170, "out/1808.05245.Vc1.Tch170.dat");

      dNpipm/dy     dV/dy[fm3]        Vc[fm3]            d/p          He3/p         H3La/p          He4/p
       0.155843              1              1    0.000121572    6.93548e-09    3.58469e-09    3.20659e-13
       0.228961        1.39317        1.39317    0.000154338    1.14019e-08    6.27757e-09    6.35122e-13
       0.342221        1.94091        1.94091    0.000201594    1.99102e-08    1.14085e-08    1.38302e-12
       0.522145        2.70402        2.70402    0.000270353     3.6667e-08    2.14172e-08    3.29281e-12
       0.814642        3.76715        3.76715    0.000371248    7.07102e-08    4.13367e-08    8.48472e-12
        1.29622        5.24826        5.24826    0.000520053    1.41691e-07    8.15378e-08    2.33455e-11
        2.08113        7.31171        7.31171    0.000738742    2.91922e-07    1.62884e-07    6.74105e-11
        3.30782        10.1864        10.1864     0.00105366    6.08188e-07    3.24761e-07    1.99431e-10
        5.11027        14.1914        14.1914 